In [0]:
file_type = 'csv'
first_row_header = 'true'
delimiter = ','
aws_keys_df = spark.read.option('header', first_row_header).csv('/FileStore/tables/new_user_credentials_2.csv')

In [0]:
from pyspark.sql.functions import *
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks').select('Access key ID').collect()[0][0]
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks').select('Secret access key').collect()[0][0]

In [0]:
import urllib
ENCODE_SECRET_KEY = urllib.parse.quote(SECRET_KEY, "")

In [0]:
AWS_S3_BUCKET = "bucket-mukul"
MOUNT_NAME = '/mnt/bucket-mukul'
source_uri = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODE_SECRET_KEY, AWS_S3_BUCKET)
dbutils.fs.mount(source_uri, MOUNT_NAME)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
<command-1339944699177514> in <module>
      2 MOUNT_NAME = '/mnt/bucket-mukul'
      3 source_uri = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODE_SECRET_KEY, AWS_S3_BUCKET)
----> 4 dbutils.fs.mount(source_uri, MOUNT_NAME)

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    317                     exc.__context__ = None
    318                     exc.__cause__ = None
--> 319                     raise exc
    320 
    321             return f_with_exception_handling

ExecutionError: An error occurred while calling o311.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/bucket-mukul; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/bucket-mukul
	at com.databrick

In [0]:
%fs ls "/mnt/bucket-mukul"

path,name,size
dbfs:/mnt/bucket-mukul/adform.click.2017.01.json,adform.click.2017.01.json,4957697816


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Emr').getOrCreate()
df = spark.read.json('/mnt/bucket-mukul/adform.click.2017.01.json')


In [0]:
df.show(n=5)

+-------------+------------+-------------+-------------+------------+-------------+--------------------+-------------+------------+--------------------+---+
|           c0|          c1|           c2|           c3|          c4|           c5|                  c6|           c7|          c8|                  c9|  l|
+-------------+------------+-------------+-------------+------------+-------------+--------------------+-------------+------------+--------------------+---+
|[-1664374510]|[1292560685]| [1963151207]| [-113426919]|[1024827180]|         null|[-1841755489, -20...|  [781804810]| [677061876]|[-2054476127, 128...|  0|
| [1566608579]|[-248982458]|  [336746857]|[-1629610286]| [244157766]|         null|[-574085389, 1869...| [1065163157]| [332083152]|        [-614983515]|  0|
| [1935105702]|[1292560685]|[-1389162932]| [-113426919]|  [-8361123]|  [839761088]|                null|[-1708330775]|[1856995055]|[-1954958362, 157...|  0|
| [1718276659]| [630920017]| [1171414431]| [-113426919]| [

In [0]:
df = df.selectExpr("coalesce(c0[0],0) as f0",
                       "coalesce(c1[0],0) as f1",
                       "coalesce(c2[0],0) as f2",
                       "coalesce(c3[0],0) as f3",
                       "coalesce(c4[0],0) as f4",
                       "l as click")



In [0]:
df.show(n=5)

+-----------+----------+-----------+-----------+----------+-----+
|         f0|        f1|         f2|         f3|        f4|click|
+-----------+----------+-----------+-----------+----------+-----+
|-1664374510|1292560685| 1963151207| -113426919|1024827180|    0|
| 1566608579|-248982458|  336746857|-1629610286| 244157766|    0|
| 1935105702|1292560685|-1389162932| -113426919|  -8361123|    0|
| 1718276659| 630920017| 1171414431| -113426919| 640993460|    1|
| 1562430026| 630920017| 1639152385| 1781226914|1493440023|    0|
+-----------+----------+-----------+-----------+----------+-----+
only showing top 5 rows



In [0]:
#Now we do something quite Spark specific, which is to reshuffle the different portions of the CSV 
#into different machines and cache them in memory
df = df.repartition(100).cache()


In [0]:
df.select('f0').distinct().count()

Out[37]: 2497

In [0]:
df.select('f1').distinct().count()

Out[38]: 178

In [0]:
df.select('f2').distinct().count()

Out[39]: 377

In [0]:
df.select('f3').distinct().count()

Out[40]: 68

In [0]:
df.select('f4').distinct().count()

Out[41]: 17572

In [0]:
from pyspark.ml.feature import StringIndexer
string_indexer = StringIndexer(inputCol="f0", outputCol='f0_index')
string_index_model = string_indexer.fit(df)
ctr_df_indexed = string_index_model.transform(df).select('f0','f0_index')
ctr_df_indexed.show(5)

+-----------+--------+
|         f0|f0_index|
+-----------+--------+
|   93008046|   864.0|
| -832245159|    17.0|
| 1771347571|   314.0|
|  830192262|  1171.0|
|-1692165568|    71.0|
+-----------+--------+
only showing top 5 rows



In [0]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [0]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCol="f0_index", outputCol="f0_encoded")
encoder_model = encoder.fit(ctr_df_indexed)

In [0]:
encoder_model.transform(ctr_df_indexed).distinct().show(5)

+-----------+--------+------------------+
|         f0|f0_index|        f0_encoded|
+-----------+--------+------------------+
|-1156005499|   337.0|(2496,[337],[1.0])|
| 1707433888|   388.0|(2496,[388],[1.0])|
|-1910840705|   118.0|(2496,[118],[1.0])|
| 1723450049|   768.0|(2496,[768],[1.0])|
| -293118980|   401.0|(2496,[401],[1.0])|
+-----------+--------+------------------+
only showing top 5 rows



In [0]:
# built a function that builds all the indexing and encoding stages that are necessary for our pipeline

def categorical_onehot_encoding_stages(columns):
    indexer = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid='keep') for column in columns]
    encoder = [OneHotEncoder(inputCol=column+"_index", outputCol=column+"_encoded") for column in columns]
    return indexer+encoder


categorical_columns = ['f0','f1','f2','f3','f4']
encoded_columns = [column +"_encoded" for column in categorical_columns]
categorical_stages = categorical_onehot_encoding_stages(categorical_columns)
vector_assembler = VectorAssembler(inputCols=encoded_columns, outputCol='features')
# Given the high number of one-hot-encoded values, the feature vector can be huge and make the trainer very slow or require massive amounts of memory. One way to mitigate that is to use a chi-squared feature selector. In our pipeline, we have selected the best 100 features. By best, we mean the features that have more predictive power—note how the chi-squared estimator takes both the features and the label to decide on the best features


selector = ChiSqSelector(numTopFeatures=100, featuresCol="features",
                         outputCol="selected_features", labelCol="click")

decision_tree = DecisionTreeClassifier(labelCol="click",                                       
                                       featuresCol="selected_features")

pipeline = Pipeline(stages=categorical_stages + [vector_assembler, selector, 
                                                 decision_tree])

In [0]:
train_df, test_df = df.randomSplit([0.8,0.2], seed=17)
pipeline_model = pipeline.fit(train_df)

In [0]:
#following code snippet shows the result of outputting the decision tree in text format:

print(pipeline_model.stages[-1].toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_b720c8413309, depth=5, numNodes=11, numClasses=2, numFeatures=100
  If (feature 3 in {1.0})
   Predict: 1.0
  Else (feature 3 not in {1.0})
   If (feature 21 in {1.0})
    Predict: 1.0
   Else (feature 21 not in {1.0})
    If (feature 89 in {1.0})
     Predict: 1.0
    Else (feature 89 not in {1.0})
     If (feature 27 in {1.0})
      Predict: 1.0
     Else (feature 27 not in {1.0})
      If (feature 29 in {1.0})
       Predict: 1.0
      Else (feature 29 not in {1.0})
       Predict: 0.0



In [0]:
#Evaluating our model
# The ROC is a representation of a trade-off between true-positive rates and false-positive rates. True-positive rates describe how good a model is at predicting a positive class when the actual class is positive. True-positive rates are calculated as the ratio of true positives predicted by a model, to the sum of true positives and false negatives. False-positive rates describe how often the model predicts the positive class, when the actual class is negative. False-positive rates are calculated as the ratio of false positives, to the sum of false positives and true negatives. ROC is a plot where the x axis is represented by the false-positive rate with a range of 0-1, while the y axis is represented as the true-positive rate. Area under curve (AUC) is the measure of the area under the ROC curve. AUC is a measure of predictiveness of a classification model.


test_transformed = pipeline_model.transform(test_df)


In [0]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='click')
evaluator.evaluate(test_transformed, {evaluator.metricName: "areaUnderROC"})

Out[50]: 0.4398949798626569

In [0]:
display(train_df)

f0,f1,f2,f3,f4,click
-2139870058,856005849,-1389162932,0,-1061368695,0
-2139870058,856005849,-1389162932,0,-1061368695,0
-2139870058,856005849,-1389162932,0,-1061368695,0
-2139870058,856005849,-1389162932,0,-1061368695,0
-2139870058,856005849,-1389162932,1781226914,-1061368695,0
-2139870058,856005849,-327095155,-425820381,-1061368695,0
-2139870058,856005849,1639152385,0,-1061368695,0
-2139870058,856005849,1639152385,0,-1061368695,0
-2139870058,856005849,1639152385,0,-1061368695,0
-2139870058,856005849,1639152385,0,-1061368695,0


In [0]:
test_transformed = pipeline_model.transform(test_df)

In [0]:
train_transformed = pipeline_model.transform(train_df)

In [0]:
#The transformed datasets includes the feature vector column (named selected_features with a size of 100). We need to transform these two columns into a CSV with 101 columns (the click and the selected_features vectors flattened out). A simple transformation in Spark allows us to do this. We define a deconstruct_vector function, which we will use to obtain a Spark dataframe with the label and each vector component as a distinct column. We then save that to S3 both for training and testing as a CSV without headers, as SageMaker requires.

#In the following code snippet, we provide the deconstruct_vector function as well as the series of transformations needed to save the dataframe:

def deconstruct_vector(row):
    arr = row['selected_features'].toArray()
    return tuple([row['click']] + arr.tolist())

df_for_csv = train_transformed.select("click", "selected_features") \
                .rdd.map(deconstruct_vector).toDF() 



---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-1339944699177538> in <module>
     10                 .rdd.map(deconstruct_vector).toDF()
     11 
---> 12 df_for_csv.write.csv("s3://bucket-mukul/", 
     13                      header=False)

/databricks/spark/python/pyspark/sql/readwriter.py in csv(self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, charToEscapeQuoteEscaping, encoding, emptyValue, lineSep)
   1397                        charToEscapeQuoteEscaping=charToEscapeQuoteEscaping,
   1398                        encoding=encoding, emptyValue=emptyValue, lineSep=lineSep)
-> 1399         self._jwrite.csv(path)
   1400 
   1401     def orc(self, path, mode=None, partitionBy=None, compression=None):

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py

In [0]:
df_for_csv.write.csv("/mnt/bucket-mukul/data", 
                     header=False)